### First Assignment - Regression Problem - Group 1



In [3]:
#Import all necessary libraries
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score

In [4]:
# Load the dataset
column_names = ['Voltage', 'Output_Air_Temperature']
HD_df = pd.read_csv(os.path.abspath('hairdryer.csv'), names=column_names)
column_names = column_names[:-1]

# Normalize the dataset
scaler_input = MinMaxScaler()
scaler_output = MinMaxScaler()

HD_df['Voltage_normalized'] = scaler_input.fit_transform(HD_df[['Voltage']])
HD_df['Output_Air_Temperature_normalized'] = scaler_output.fit_transform(HD_df[['Output_Air_Temperature']])

# Create the x and y variables
x = HD_df['Voltage_normalized'].values.reshape(-1, 1)
y = HD_df['Output_Air_Temperature_normalized'].values.reshape(-1, 1)

# Split the variables into train and test sets
seed = 7
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=seed)

# Squeeze y_train to correct the dimensions
y_train = np.squeeze(y_train)

In [5]:
# Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=3)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')

In [6]:
# Estimate membership functions parameters
ae = AntecedentEstimator(X_train, part_matrix)
antecedent_params = ae.determineMF()

In [7]:
# Estimate consequent parameters
ce = ConsequentEstimator(X_train, y_train, part_matrix)
conseq_params = ce.suglms()

In [8]:
# Build first-order Takagi-Sugeno model
modbuilder = SugenoFISBuilder(antecedent_params, conseq_params, column_names, save_simpful_code=False)
model = modbuilder.get_model()

 * Detected 3 rules / clusters
 * Detected Sugeno model type


In [9]:
# Get model predictions
modtester = SugenoFISTester(model, X_test, column_names)
y_pred = modtester.predict()[0]

In [10]:
# Perform the inverse transform 
y_pred_original = scaler_output.inverse_transform(y_pred.reshape(-1, 1))
y_test_original = scaler_output.inverse_transform(y_test.reshape(-1, 1))

In [43]:
# Compute regression metrics
mse = mean_squared_error(y_test_original, y_pred_original)
print("Mean Squared Error: {:.3f}".format(mse))
mape = mean_absolute_percentage_error(y_test_original, y_pred_original)
print("Mean Absolute Percentage Error: {:.1f}%".format(mape*100))
exp_var = explained_variance_score(y_test_original, y_pred_original)
print("Explained Variance Score: {:.3f}".format(exp_var))

Mean Squared Error: 0.674
Mean Absolute Percentage Error: 15.6%
Explained Variance Score: 0.016
